In [ ]:
import matplotlib
matplotlib.use("Agg")
%matplotlib inline 

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

from keras import backend as K

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,Nadam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from vggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

#construct argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d","--data",required = True,
#                 help = "path to input dataset")
# ap.add_argument("-m","--model",required = True,
#                  help = "path to model")
# ap.add_argument("-l","--labelbin",required=True,
#                 help = "path to output label binarizer")
# ap.add_argument("-p","--plot",type=str,default="plot.png",
#                 help="path to output accuracy vs loss plot")
# args = vars(ap.parse_args())
#initialize epochs,learning rate,batch size and img dims
epochs = 100
lr = 0.0002
bs = 64
IMAGE_DIMS = (128,128,3)
#initialize data and labels
data = []
labels = []
print("[INFO] loading images ...")
imagePaths = sorted(list(paths.list_images('data')))
random.seed(31)
random.shuffle(imagePaths)
f = 1
#loop over input images
for imagePath in imagePaths:
    #load the image,preprocess it, and store it in data list
    image = cv2.imread(imagePath)
    if image is not None:
        image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)

        #extract class label; from the image path and update labels list
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)
        f+=1
    
print(f)
#scale pixel intensities from 0-255 to 0-1
data = np.array(data,dtype="float")/255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(data.nbytes/(1024*1000.0)))
#binarize labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels)
#partition the data into training and testing splits using 80%
#of the data for trainning and the remaining 20% for testing
(trainX,testX,trainY,testY) = train_test_split(data,labels,
    test_size=0.08,random_state = 31)
"""Data Augmentation is IMP cos low data"""
#construct image generator for data augmentation
aug = ImageDataGenerator(rotation_range = 25,width_shift_range=0.1,
                         height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,
                         horizontal_flip=True,fill_mode="nearest")
#initialize the model
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1],height=IMAGE_DIMS[0],
                            depth=IMAGE_DIMS[2],classes=len(lb.classes_))
opt = Nadam(lr=lr)
#categorical crossentropy as multi-class output
#for 2 classes outputs, use binary crossentropy
model.compile(loss="categorical_crossentropy",optimizer=opt,
              metrics=["accuracy"])

#train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX,trainY,batch_size=bs),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX)//bs,
                        epochs=epochs,verbose=1)

In [ ]:
#save model
print("[INFO] serializing network...")
model.save('vehicle0.model')
#save label binarizer to disk
print("[INFO] serializing label binarizer...")
f=open('lb.pickle',"wb")
f.write(pickle.dumps(lb))
f.close()

 

In [ ]:
plt.style.use("ggplot")
plt.figure()
n = epochs
plt.plot(np.arange(0,n),H.history["loss"],label="train_loss")
plt.plot(np.arange(0,n),H.history["val_loss"],label="val_loss")
plt.plot(np.arange(0,n),H.history["acc"],label = "trian_acc")
plt.plot(np.arange(0,n),H.history["val_acc"],label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig('plot.png')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec  9 22:35:46 2018

@author: servo97
"""

from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

ap = argparse.ArgumentParser()
'''ap.add_argument("-m","--model", required = True,
                help="path to trained model")
ap.add_argument("-l","--labelbin",required = True,
                help="path to label binarizer")
ap.add_argument("-i","--image",required=True,
                help="path to input image")
args=vars(ap.parse_args())
'''
#load the model
model = load_model("vehicle0.model")
lb= pickle.loads(open("lb.pickle","rb").read())

image1 = cv2.imread("examples/10061.jpg")
op1 = image1.copy()
image2 = cv2.imread("examples/80048.jpg")
op2 = image2.copy()
image3 = cv2.imread("examples/th - 2019-03-31T221516.576.jpg")
op3 = image3.copy()

image3 = cv2.resize(image3,(128,128))
image3 = image3.astype("float")/255.0
image3 = img_to_array(image3)
image3 = np.expand_dims(image3,axis=0)

image1 = cv2.resize(image1,(128,128))
image1 = image1.astype("float")/255.0
image1 = img_to_array(image1)
image1 = np.expand_dims(image1,axis=0)

image2 = cv2.resize(image2,(128,128))
image2 = image2.astype("float")/255.0
image2 = img_to_array(image2)
image2 = np.expand_dims(image2,axis=0)

print("Classifying images...")
prob1 = model.predict(image1)[0]
idx1 = np.argmax(prob1)
label1 = lb.classes_[idx1]

prob2 = model.predict(image2)[0]
idx2 = np.argmax(prob2)
label2 = lb.classes_[idx2]

prob3 = model.predict(image3)[0]
idx3 = np.argmax(prob3)
label3 = lb.classes_[idx3]

# we'll mark our prediction as "correct" of the input image filename
# contains the predicted label text (obviously this makes the
# assumption that you have named your testing image files this way)
'''filename = args["image"][args["image"].rfind(os.path.sep) + 1:]
correct = "correct" if filename.rfind(label)!=-1 else "correct"'''

#build label and draw it on image
label1 = "{}: {:.2f}%".format(label1,prob1[idx1]*100)
op1 = imutils.resize(op1,width=400)
cv2.putText(op1,label1,(10,25),cv2.FONT_HERSHEY_SIMPLEX,0.7,(200,0,0),2)
print(label1)
%matplotlib inline
import numpy as np
import skimage
from skimage import data
from matplotlib import pyplot as plt

img1 = op1
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_RGB2BGR))
plt.show()

label2 = "{}: {:.2f}%".format(label2,prob2[idx2]*100)
op2 = imutils.resize(op2,width=400)
cv2.putText(op2,label2,(10,25),cv2.FONT_HERSHEY_SIMPLEX,0.7,(200,0,0),2)
print(label2)

img2 = op2
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))
plt.show()

label3 = "{}: {:.2f}%".format(label3,prob3[idx3]*100)
op3 = imutils.resize(op3,width=400)
cv2.putText(op3,label3,(10,25),cv2.FONT_HERSHEY_SIMPLEX,0.7,(200,0,0),2)
print(label3)

img3 = op3
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_RGB2BGR))
plt.show()
